In [16]:
%pwd  #look at the current work dir

'/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-data-pipeline'

In [17]:
%cd '/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-data-pipeline'

/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-data-pipeline


/Users/yannickflores/Library/Caches/pypoetry/virtualenvs/foreshadwownba-dvc-data-pipeline-A60HmWSY-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [18]:
%pwd  #look at the current work dir

'/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-dvc-data-pipeline'

In [ ]:
from pydantic.types import StrictInt
import pandas as pd

from pandantic import BaseModel


class DataFrameSchema(BaseModel):
    """Example schema for testing."""

    example_str: str
    example_int: StrictInt

df_invalid = pd.DataFrame(
    data={
        "example_str": ["foo", "bar", 1],
        "example_int": ["1", 2, 3.0],
    }
)

df_filtered = DataFrameSchema.parse_df(
    dataframe=df_invalid,
    errors="filter",
)

In [19]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [20]:
df = pd.read_csv("./data/processed/nba_games_training_dataset_pre_cleaned.csv")

df = df.sort_values(
        ['id_season','tm', 'game_nb']
    )

## Last Games Overtime

In [21]:
from src.feature_engineering_functions.last_game_overtime import last_game_overtime

In [22]:
df = last_game_overtime(df)

In [23]:
df.head()

,id_season,id,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,w_tot,overtime,streak_w_l,last_game_overtime
12167,2010,2009-10-28_IND_ATL,1,2009-10-28,dom,ATL,IND,W,120,109,1.0,NOT,W 1,NaN
12169,2010,2009-10-30_WAS_ATL,2,2009-10-30,dom,ATL,WAS,W,100,89,2.0,NOT,W 2,NOT
12171,2010,2009-11-01_ATL_LAL,3,2009-11-01,ext,ATL,LAL,L,110,118,2.0,NOT,L 1,NOT
12173,2010,2009-11-03_ATL_POR,4,2009-11-03,ext,ATL,POR,W,97,91,3.0,NOT,W 1,NOT
12175,2010,2009-11-04_ATL_SAC,5,2009-11-04,ext,ATL,SAC,W,113,105,4.0,NOT,W 2,NOT


In [24]:
df[(df['last_game_overtime']=='OT') & (df['id_season']==2024)].head()

,id_season,id,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,w_tot,overtime,streak_w_l,last_game_overtime
8319,2024,2023-11-25_ATL_WAS,15,2023-11-25,ext,ATL,WAS,W,136,108,8.0,NOT,W 2,OT
8360,2024,2024-01-10_PHI_ATL,36,2024-01-10,dom,ATL,PHI,W,139,132,15.0,OT,W 1,OT
8364,2024,2024-01-12_IND_ATL,37,2024-01-12,dom,ATL,IND,L,108,126,15.0,NOT,L 1,OT
8387,2024,2024-02-05_LAC_ATL,50,2024-02-05,dom,ATL,LAC,L,144,149,22.0,NOT,L 1,OT
8414,2024,2024-03-30_MIL_ATL,74,2024-03-30,dom,ATL,MIL,L,113,122,34.0,NOT,L 1,OT


In [25]:
test_ot_data = df[(df['tm']=='ATL') & (df['id_season']==2024)]
test_ot_data.head(5)

,id_season,id,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,w_tot,overtime,streak_w_l,last_game_overtime
8299,2024,2023-10-25_ATL_CHO,1,2023-10-25,ext,ATL,CHO,L,110,116,0.0,NOT,L 1,NaN
8300,2024,2023-10-27_NYK_ATL,2,2023-10-27,dom,ATL,NYK,L,120,126,0.0,NOT,L 2,NOT
8301,2024,2023-10-29_ATL_MIL,3,2023-10-29,ext,ATL,MIL,W,127,110,1.0,NOT,W 1,NOT
8302,2024,2023-10-30_MIN_ATL,4,2023-10-30,dom,ATL,MIN,W,127,113,2.0,NOT,W 2,NOT
8303,2024,2023-11-01_WAS_ATL,5,2023-11-01,dom,ATL,WAS,W,130,121,3.0,NOT,W 3,NOT


In [26]:
# Id of the game for where the team played OT
ot_game = test_ot_data[test_ot_data['overtime']=='OT']['game_nb'].values[0]

# game_nb where the overtime feature should be equal to OT
game_nb_ot = ot_game + 1

last_game_overtime_feature = test_ot_data[test_ot_data['game_nb']==game_nb_ot]['last_game_overtime']

# assert last_game_overtime_feature.values[0] == 'OT'

last_game_overtime_feature.values[0]

'OT'

## Test Calculate Streak Features

In [27]:
from src.feature_engineering_functions.games_streak_features import calculate_streak_features

In [28]:
nba_games_w_streak_games_features = calculate_streak_features(
    df
    )

test_streak_games_features = nba_games_w_streak_games_features[
    (nba_games_w_streak_games_features['tm']=='GSW') &
    (nba_games_w_streak_games_features['id_season']==2017)]


test_streak_games_features.head(6)

,id_season,id,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,w_tot,overtime,streak_w_l,last_game_overtime,streak_w_l_2,before_streak_w_l
4360,2017,2016-10-25_SAS_GSW,1,2016-10-25,dom,GSW,SAS,L,100,129,0.0,NOT,L 1,NaN,-1,NaN
4363,2017,2016-10-28_GSW_NOP,2,2016-10-28,ext,GSW,NOP,W,122,114,1.0,NOT,W 1,NOT,1,-1.0
4367,2017,2016-10-30_GSW_PHO,3,2016-10-30,ext,GSW,PHO,W,106,100,2.0,NOT,W 2,NOT,2,1.0
4371,2017,2016-11-01_GSW_POR,4,2016-11-01,ext,GSW,POR,W,127,104,3.0,NOT,W 3,NOT,3,2.0
4375,2017,2016-11-03_OKC_GSW,5,2016-11-03,dom,GSW,OKC,W,122,96,4.0,NOT,W 4,NOT,4,3.0
4379,2017,2016-11-04_GSW_LAL,6,2016-11-04,ext,GSW,LAL,L,97,117,4.0,NOT,L 1,NOT,-1,4.0


In [29]:
# Id of the game for where the team played OT
streak_w_l_game = test_streak_games_features[test_streak_games_features['streak_w_l']=='W 4']['game_nb'].values[0]

# game_nb where the overtime feature should be equal to OT
streak_w_l_game = streak_w_l_game + 1

last_streak_w_l_game_feature = test_streak_games_features[test_streak_games_features['game_nb']==streak_w_l_game]['before_streak_w_l']

# assert last_game_overtime_feature.values[0] == 4

last_streak_w_l_game_feature.values[0]

np.float64(4.0)

## Test Games date processing

In [30]:
from src.feature_engineering_functions.games_date_processing import (
    extract_days_of_week_from_date,
    game_on_weekend_features,
)

In [33]:
nba_games_w_days_of_week_from_date = extract_days_of_week_from_date(
    df
    )

nba_games_w_days_of_week_from_date = nba_games_w_days_of_week_from_date[
    (nba_games_w_days_of_week_from_date['tm']=='MIA') &
    (nba_games_w_days_of_week_from_date['id_season']==2012)]


nba_games_w_days_of_week_from_date.head(6)

,id_season,id,game_nb,game_date,extdom,tm,opp,results,pts_tm,pts_opp,w_tot,overtime,streak_w_l,last_game_overtime,day_of_week
24373,2012,2011-12-25_MIA_DAL,1,2011-12-25,ext,MIA,DAL,W,105,94,1.0,NOT,W 1,NaN,Sunday
24375,2012,2011-12-27_BOS_MIA,2,2011-12-27,dom,MIA,BOS,W,115,107,2.0,NOT,W 2,NOT,Tuesday
24379,2012,2011-12-28_MIA_CHA,3,2011-12-28,ext,MIA,CHA,W,96,95,3.0,NOT,W 3,NOT,Wednesday
24381,2012,2011-12-30_MIA_MIN,4,2011-12-30,ext,MIA,MIN,W,103,101,4.0,NOT,W 4,NOT,Friday
24388,2012,2012-01-01_CHA_MIA,5,2012-01-01,dom,MIA,CHA,W,129,90,5.0,NOT,W 5,NOT,Sunday
24390,2012,2012-01-02_ATL_MIA,6,2012-01-02,dom,MIA,ATL,L,92,100,5.0,NOT,L 1,NOT,Monday


In [48]:
day_of_week_values = nba_games_w_days_of_week_from_date['day_of_week'].value_counts()
week_days_list = sorted(list(day_of_week_values.index))

In [49]:
ref_list = sorted(['Sunday', 'Tuesday', 'Friday', 'Wednesday', 'Thursday', 'Monday', 'Saturday'])

week_days_list == ref_list

True

In [53]:
nba_games_w_days_of_week_from_date[nba_games_w_days_of_week_from_date['game_date']=='2012-01-01']['day_of_week'].values[0] == 'Sunday'

True